## 1. Importar Librerías

Usamos el SDK de Azure Cognitive Services Speech.

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk
from dotenv import load_dotenv

## 2. Cargar Credenciales

Necesitamos dos valores del archivo `.env`:
- `SPEECH_KEY`: La clave de API del servicio de Speech
- `SPEECH_REGION`: La región donde está desplegado el servicio

In [ ]:
load_dotenv()

speech_key = os.getenv("SPEECH_KEY")
service_region = os.getenv("SPEECH_REGION")

if not speech_key or not service_region:
    raise RuntimeError("Faltan SPEECH_KEY y/o SPEECH_REGION en el archivo .env")

print(f"✅ Región configurada: {service_region}")
print(f"✅ Key configurada: {'*' * 10}{speech_key[-4:]}")

## 3. Configurar el Servicio de Speech

Configuramos:
- La clave y región del servicio
- El idioma de reconocimiento (español en este caso)

In [ ]:
speech_config = speechsdk.SpeechConfig(
    subscription=speech_key, 
    region=service_region
)

# Configurar idioma a español de España
speech_config.speech_recognition_language = "es-ES"

print("✅ Configuración de Speech creada")
print(f"   Idioma: {speech_config.speech_recognition_language}")

## 4. Idiomas Disponibles

Azure Speech soporta múltiples idiomas. Aquí algunos ejemplos:

In [ ]:
idiomas_comunes = {
    "es-ES": "Español (España)",
    "es-MX": "Español (México)",
    "en-US": "English (United States)",
    "en-GB": "English (United Kingdom)",
    "fr-FR": "Français (France)",
    "de-DE": "Deutsch (Deutschland)",
    "it-IT": "Italiano (Italia)",
    "pt-BR": "Português (Brasil)",
    "ja-JP": "日本語 (日本)",
    "zh-CN": "中文 (简体)",
}

print("🌍 Idiomas soportados (ejemplos):")
for code, name in idiomas_comunes.items():
    print(f"   {code}: {name}")

## 5. Configurar la Entrada de Audio

Configuramos el sistema para usar el micrófono predeterminado del sistema.

In [ ]:
audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

print("✅ Audio configurado para usar micrófono predeterminado")

## 6. Crear el Reconocedor de Voz

Combinamos la configuración de speech y audio para crear el reconocedor.

In [ ]:
speech_recognizer = speechsdk.SpeechRecognizer(
    speech_config=speech_config, 
    audio_config=audio_config
)

print("✅ Reconocedor de voz creado y listo")

## 7. Realizar Reconocimiento de Voz

Ejecuta esta celda y habla en español cuando veas el mensaje.

**Importante**: 
- Asegúrate de que tu micrófono esté funcionando
- Habla claramente
- El reconocimiento se detiene automáticamente después de una pausa

In [ ]:
print("\n" + "="*80)
print("🎤 RECONOCIMIENTO DE VOZ INICIADO")
print("="*80)
print("\n👉 Habla algo en español...\n")

# Realizar reconocimiento único
result = speech_recognizer.recognize_once()

print("\n" + "─"*80)

## 8. Procesar el Resultado

El resultado puede tener diferentes estados:
- `RecognizedSpeech`: Éxito, se reconoció el texto
- `NoMatch`: No se pudo reconocer
- `Canceled`: El reconocimiento fue cancelado (error)

In [ ]:
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("\n✅ RECONOCIMIENTO EXITOSO")
    print("\n📝 Texto reconocido:")
    print(f"\n    \"{result.text}\"\n")
    
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("\n❌ NO SE PUDO RECONOCER LA VOZ")
    print("\nPosibles razones:")
    print("   - El audio era muy bajo")
    print("   - Había mucho ruido de fondo")
    print("   - No se detectó habla")
    print(f"\nDetalles: {result.no_match_details}")
    
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("\n⚠️ RECONOCIMIENTO CANCELADO")
    print(f"\nRazón: {cancellation_details.reason}")
    
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("\n❌ ERROR DETECTADO:")
        if cancellation_details.error_details:
            print(f"   {cancellation_details.error_details}")
        print("\nVerifica:")
        print("   - SPEECH_KEY es correcta")
        print("   - SPEECH_REGION es correcta")
        print("   - Tienes conexión a internet")

print("\n" + "="*80)

## 9. Función Reutilizable

Creamos una función para facilitar el reconocimiento de voz.

In [ ]:
def recognize_speech(language="es-ES"):
    """Reconoce voz desde el micrófono."""
    # Configurar
    config = speechsdk.SpeechConfig(
        subscription=speech_key, 
        region=service_region
    )
    config.speech_recognition_language = language
    
    audio = speechsdk.audio.AudioConfig(use_default_microphone=True)
    recognizer = speechsdk.SpeechRecognizer(
        speech_config=config, 
        audio_config=audio
    )
    
    # Reconocer
    print(f"🎤 Escuchando ({language})...")
    result = recognizer.recognize_once()
    
    # Procesar
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        return None
    else:
        raise Exception(f"Error: {result.cancellation_details.reason}")

print("✅ Función recognize_speech() definida")

## 10. Probar la Función (Opcional)

Descomenta y ejecuta para probar la función.

In [ ]:
# texto = recognize_speech("es-ES")
# if texto:
#     print(f"✅ Reconocido: {texto}")
# else:
#     print("❌ No se reconoció nada")

print("💡 Descomenta el código anterior para probar la función")

## 11. Reconocimiento en Otros Idiomas

Podemos cambiar fácilmente el idioma de reconocimiento.

In [ ]:
# Ejemplo con inglés
# texto_en = recognize_speech("en-US")
# print(f"English: {texto_en}")

# Ejemplo con francés
# texto_fr = recognize_speech("fr-FR")
# print(f"Français: {texto_fr}")

print("💡 Descomenta para probar otros idiomas")

## Conclusión

Este ejemplo demuestra:

### 1. **Azure Speech Service**
- Reconocimiento de voz en tiempo real
- Soporte para múltiples idiomas
- Alta precisión en el reconocimiento

### 2. **Configuración Simple**
- Solo necesitas key y región
- Configuración de idioma flexible
- Fácil cambio entre idiomas

### 3. **Manejo de Errores**
- Distintos estados de resultado
- Mensajes de error claros
- Fácil depuración

### Aplicaciones Prácticas:
- **Asistentes de voz**: Siri, Alexa, Google Assistant
- **Transcripción**: Reuniones, entrevistas, conferencias
- **Subtitulado**: Videos en tiempo real
- **Accesibilidad**: Interfaces para personas con discapacidades
- **Dictado**: Aplicaciones de escritura por voz
- **Call centers**: Transcripción de llamadas

### Características Avanzadas:
Azure Speech también soporta:
- **Reconocimiento continuo**: Para audio largo
- **Custom Speech**: Entrenar modelos personalizados
- **Identificación de hablantes**: Distinguir entre personas
- **Traducción de voz**: Speech-to-Speech translation
- **Text-to-Speech**: Convertir texto a voz

### Notas Importantes:
1. **Privacidad**: El audio se procesa en Azure, no localmente
2. **Costos**: El servicio tiene costos por minuto de audio procesado
3. **Latencia**: Requiere conexión a internet estable
4. **Calidad**: El ruido de fondo afecta la precisión